<div >
<img src = "figs/ans_banner_1920x200.png" />
</div>

# Caso-taller:  Recomendando el Blog de  Hernán Casciari 


[Hernán Casciari](https://hernancasciari.com/#bio), es un escritor argentino, que escribe blog posts con cuentos e historias  relacionadas con el futbol, su vida, infancia, y relaciones familiares con toques de ficción. Este [blog](https://hernancasciari.com/blog/) es  tan interesantes que en 2005 fue premiado como “El mejor blog del mundo” por Deutsche Welle de Alemania. 

El objetivo de este caso-taller es construir un sistema de recomendación basado en los contenidos de los posts utilizando similitud de las palabras usadas o temas de los cuentos.

## Instrucciones generales

1. Para desarrollar el *cuaderno*, primero debe descargarlo.

2. Para responder cada inciso deberá utilizar el espacio debidamente especificado.

3. La actividad será calificada sólo si sube el *cuaderno* de jupyter notebook con extensión `.ipynb` en la actividad designada como "entrega calificada por el personal".

4. El archivo entregado debe poder ser ejecutado localmente por el tutor. Sea cuidadoso con la especificación de la ubicación de los archivos de soporte, guarde la carpeta de datos en el mismo `path` de su cuaderno, por ejemplo: `data`.

## Desarrollo


### 1. Carga de datos 

En la carpeta `data` se encuentran el archivo `blog_casciari.csv` con el título, la fecha de publicación, y el contenido de los cuentos publicados en el blog  de sr. Casciari. Cargue estos datos en su *cuaderno* y reporte brevemente el contenido de la base.
   

In [ ]:
import pandas as pd
df = pd.read_csv('blog_casciari.csv')

: 

In [ ]:
df.shape

: 

In [ ]:
df.head()

: 

In [ ]:
df.info()

In [ ]:
import matplotlib.pyplot as plt

df['fecha'] = pd.to_datetime(df['fecha'], format='%m/%d/%y')
df['año'] = df['fecha'].dt.year
conteo_por_año = df['año'].value_counts().sort_index()
plt.figure(figsize=(10, 6))
plt.barh(conteo_por_año.index, conteo_por_año.values)
plt.xlabel('Número de observaciones')
plt.ylabel('Año')
plt.title('Número de observaciones por año (Gráfico Horizontal)')

plt.yticks(conteo_por_año.index)

plt.show()

Para comenzar, se almacenan los datos del blog en un data frame de pandas llamado df a través de la función read_csv. A continuación se verifican las dimensiones del data frame, de donde se obtienen 520 filas y 3 columnas. Es decir que el blog contiene 520 cuentos. A través de head se ven las primeras observaciones y con info se verifica que no hay valores nulos en la base de datos. También se hace un gráfico para verificar el año de escritura de los cuentos. Acá se concluye que el año en el que más cuentos se escribieron en el blog fue en 2004.

### 2. Homogenización de textos

Para cumplir con el objetivo de generar recomendaciones en esta sección debe preparar los posts para poder ser utilizados en su sistema de recomendación. Para ello, "limpie" y "tokenize" cada uno de los cuentos, describiendo detalladamente los pasos que realizo y si transformó o eliminó ciertas palabras. Para asistirlo en la tarea he creado listas de *stopwords* que están disponibles en la carpeta `data`. En su procedimiento ilustre la limpieza con el cuento 'La venganza del metegol'. (En su limpieza recuerde que el objetivo es generar recomendaciones a partir de la similitud de las palabras o temas de los cuentos)

Para comenzar, se mostrará el paso a paso con el cuento "La venganza del metegol" y a continuación se realizará todo el procedimiento a partir de una sola función para todos los cuentos. En primer lugar, se verifica la posición en la que se encuentra este cuento. De este modo, se verifica que se encuentra en la fila 160.

In [ ]:
df[df['titulo'] == "La venganza del metegol"]

Con esta información, se genera un objeto llamado metegol que contiene el cuento solicitado y se muestra su texto.

In [ ]:
metegol = df['cuento'][160]
metegol

Se importan las librerías re para el tratamiento de las expresiones regulares y unidecode para eliminar caracteres no deseados transformando el texto en ASCII a través de la función unidecode. A continuación se muestran los primeros 1000 caracteres del texto, donde se verifica que se eliminaron tildes y virgulillas.

In [ ]:
import re
import unidecode

metegol = unidecode.unidecode(metegol)
metegol[0:1000]

Con la función sub se trabajan las expresiones regulares. ^\w\s es una clase de caracteres negada que busca cualquier carácter que no sea una letra o un dígito (\w) ni un espacio en blanco (\s). En otras palabras, busca cualquier carácter que no sea una letra, un dígito o un espacio en blanco. El símbolo | actúa como un operador OR, lo que significa que estamos buscando cualquier carácter que coincida con el patrón anterior (cualquier carácter que no sea una letra, un dígito o un espacio en blanco) o el siguiente patrón. \n representa el carácter de nueva línea en una cadena de texto. Todos estos patrones se reemplazan por espacios en blanco.

In [ ]:
metegol = re.sub("[^\\w\\s]|\n", ' ', metegol)
print(metegol[0:1000])

En el código que se muestra a continuación se eliminan los dígitos del texto por espacios en blanco.

In [ ]:
metegol = re.sub("\d+", "", metegol)
print(metegol[0:1000])

A continuación, se eliminan los espacios extra:

In [ ]:
metegol = re.sub('\s+', ' ', metegol)
print(metegol[0:1000])

Y se pasa todo el texto a minúsculas:

In [ ]:
metegol =  metegol.lower()
print(metegol[0:1000])

A continuación se importa spacy, que es una librería que será usada para la lematización y la tokenización. La línea de código nlp = spacy.load("es_core_news_sm") se utiliza para cargar un modelo de procesamiento de lenguaje natural (NLP) pre-entrenado para el español. El modelo es guardado en el objeto nlp.

In [ ]:
#pip install spacy

In [ ]:
import spacy
nlp = spacy.load("es_core_news_sm")

Este modelo pre entrenado es aplicado al cuento elegido previamente:

In [ ]:
metegol = nlp(metegol)
print(metegol[0:1000])

Para eliminar los stopwords se usan los suministrados por el equipo docente para este ejercicio, añadiéndolas a las stopwords predefinidas en el modelo cargado previamente. Estas están contenidas en extra_stopwords y stopwords_taller.

In [ ]:
extra_stopwords = pd.read_csv('extra_stopwords.csv', sep=',',header=None)
extra_stopwords.columns = ['stopwords']
extra_stopwords=set(extra_stopwords['stopwords'].to_list()

In [ ]:
stopwords_taller = pd.read_csv('stopwords_taller.csv', sep=',',header=None)
stopwords_taller.columns = ['stopwords']
stopwords_taller=set(stopwords_taller['stopwords'].to_list())

In [ ]:
nlp.Defaults.stop_words |= extra_stopwords
nlp.Defaults.stop_words |= stopwords_taller

Así, se conservan solamente las palabras que no están en la lista de stopwords. De aquí se evidencia que se elimina la palabra "Buenos" que quizás convendría conservar debido a que en la literatura hispanoamericana es muy habitual que en la narrativa, el espacio geográfico sea protagónico, por lo cual las ciudades podrían incluirse en los sistemas de recomendación y la eliminación de esta palabra excluye "Buenos Aires".

In [ ]:
metegol = [token.text for token in metegol if not token.is_stop]
metegol = " ".join(metegol)
print(metegol[0:1000])

A continuación, se realiza la lematización. En esta, se observa que hay algunas palabras que no se lematizan correctamente. Por ejemplo, charla se convierte en char él.

In [ ]:
lemmas =[token.lemma_ for token in nlp(metegol)]

metegol = " ".join(lemmas)
print(metegol[0:1000])

Debido a que muchas veces se ha observado que, en la lematización se genera el término "él", se elimina este del texto:

In [ ]:
metegol = re.sub('él', '', metegol)
print(metegol[0:1000])

Se genera entonces una lista de palabras, pero solamente aquellas palabras con más de 2 caracteres:

In [ ]:
metegol = [token.text for token in nlp(metegol) if len(token) > 2]
print(metegol[0:1000])

Todos estos procedimientos se resumen en una función, dejando la aclaración que esta función asume que previamente se han agregado las stop_words suministradas:

In [ ]:
#extra_stopwords = pd.read_csv('extra_stopwords.csv', sep=',',header=None)
#extra_stopwords.columns = ['stopwords']
#extra_stopwords=set(extra_stopwords['stopwords'].to_list())
#stopwords_taller = pd.read_csv('stopwords_taller.csv', sep=',',header=None)
#stopwords_taller.columns = ['stopwords']
#stopwords_taller=set(stopwords_taller['stopwords'].to_list())
#nlp.Defaults.stop_words |= extra_stopwords
#nlp.Defaults.stop_words |= stopwords_taller
def text_cleaning(txt):
    
    # Eliminar caracteres especiales
    out = unidecode.unidecode(txt)
    out = re.sub("[^\\w\\s]|\n", ' ', out)
    out = re.sub("\d+", "", out)
    out = re.sub('\s+', ' ', out)
    # Poner en minúsculas
    out = out.lower()
    #NLP object
    out = nlp(out)
    # Eliminar Stopwords
    out = [token.text for token in out if not token.is_stop]
    out = " ".join(out)
    # Obtener los lemas de cada palabra
    lemmas =[token.lemma_ for token in nlp(out)]
    # Convertir la lista de lemmas nuevamente a texto
    out = " ".join(lemmas)
    # Remover "él"
    out = re.sub('él', '', out)
    # Remover palabras muy cortas
    out = [token.text for token in nlp(out) if len(token) > 2]
    
    return out

La limpieza definida en esta función es aplicada a cada uno de los cuentos, que son almacenados ya limpios en una lista. Con esta se unen los tokens en un objeto llamado clean_sentences en donde cada línea es un cuento. La línea 160 muestra el cuento "La venganza del metegol" con todo el proceso de limpieza.

In [ ]:
clean = list(map(text_cleaning, df['cuento']))

clean_sentences = [" ".join(i) for i in clean]

# Vemos la linea 100 limpia
print(clean_sentences[160])

### 3. Generando Recomendaciones

En esta sección nos interesa generar recomendaciones de cuentos en el blog a un usuario que leyó 'La venganza del metegol'. Para ello vamos a utilizar distintas estrategias.

#### 3.1. Recomendaciones basadas en contenidos

##### 3.1.1. Genere 5 recomendaciones de más recomendada (1) a menos recomendada (5) para el cuento 'La venganza del metegol' usando en la distancia de coseno donde el texto este vectorizado por `CountVectorizer`. Explique el procedimiento que realizó y como ordenó las recomendaciones.

Se importa CountVectorizer de sklearn y el modelo es guardado en un objeto llamado count. Este es aplicado a clean_sentences, de donde se genera una matriz con 520 filas y 24622 columnas (términos)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()
count_matrix = count.fit_transform(clean_sentences)
count_matrix.shape

Esta matriz se convierte a un formato denso almacenado en un data frame denominado df_count que contiene el mismo número de filas y columnas que la matriz anterior.

In [ ]:
df_count = pd.DataFrame(count_matrix.toarray(), columns=count.get_feature_names_out())
df_count.head()

A partir de esta función se identifican algunos términos que no fueron lematizados correctamente, por lo cual se crea la función Lematizador_propio para abandonar, abalanzar, zurdo, zurrar y zozobrar:

In [ ]:
def Lematizador_propio(text):
    # Diccionario con las palabras y sus lemas correspondientes
    lemmas = {
        r"\babandona\b": "abandonar",
        r"\babandonado\b": "abandonar",
        r"\babandonandolo\b": "abandonar",
        r"\babandonar\b": "abandonar",
        r"\babandono\b": "abandonar",
        r"\babandón\b": "abandonar",
        r"\babalanzo\b": "abalanzar",
        r"\bzurda\b": "zurdo",
        r"\bzurdazo\b": "zurdo",
        r"\bzurdito\b": "zurdo",
        r"\bzurraba\b": "zurrar",
        r"\bzurrartir\b": "zurrar",
        r"\bzozobra\b": "zozobrar"
    }
    for pattern, lemma in lemmas.items():
        text = re.sub(pattern, lemma, text, flags=re.IGNORECASE)
    return text

clean_sentences2 = list(map(Lematizador_propio, clean_sentences))

Después de aplizar el nuevo lematizador, se aplica de nuevo el conteo de términos con CountVectorizer, disminuyendo el número de columnas.

In [ ]:
count = CountVectorizer()
count_matrix = count.fit_transform(clean_sentences2)

df_count = pd.DataFrame(count_matrix.toarray(), columns=count.get_feature_names_out())
df_count.head()

Se importa linear_kernel con el fin de calcular la distancia de coseno en la matriz count_matrix

In [ ]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(count_matrix, count_matrix)

Ahora, se define la función de recomendador donde se extraen los índices, se identifica el índice del cuento requerido por la función en la opción titulo y se extraen los puntajes de similitud de coseno aplicados a este cuento. En el paso 6 se extraen los primeros 5 cuentos con mayor puntaje en la similitud de coseno que fueron ordenados en el paso 5. No se incluye el elemento 0 de la lista debido a que este corresponde al mismo cuento solicitado en la función.La función retorna los títulos de los cuentos similares.

In [ ]:
def recomendador(titulo, cosine_sim=cosine_sim, df=df):
    #Paso 2
    df = df.reset_index()
    indices = pd.Series(df.index, index=df['titulo']).drop_duplicates()
    #Paso 3
    idx = indices[titulo]

    #Paso 4
    sim_scores = list(enumerate(cosine_sim[idx]))

    #Paso 5
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    #Paso 6
    sim_scores = sim_scores[1:6]

    cuento_indices = [i[0] for i in sim_scores]

    #Paso 7
    return df['titulo'].iloc[cuento_indices]

Se prueba la función y se obtienen los 5 cuentos recomendados.

In [ ]:
recomendador("La venganza del metegol")

A través de una nube de palabras se obtienen los términos más usados en estos cuentos. De aquí se ve que los términos están relacionados con la visión y la fotografía. Aun así, hay algunas palabras muy repetidas que quizás no son tan informativos como por ejemplo año.

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

cuentos_rec=df[df['titulo'].isin(recomendador("La venganza del metegol"))]

text = " ".join(cuentos_rec['cuento'])
text = text_cleaning(text)
text=' '.join(text)
text = Lematizador_propio(text)

wordcloud = WordCloud(width = 1600, height = 800, 
    background_color = "white").generate(text)
plt.figure(figsize = (20, 10))
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis("off")
plt.show()

##### 3.1.2. Genere 5 recomendaciones de más recomendada (1) a menos recomendada (5) para  el cuento 'La venganza del metegol' usando nuevamente la distancia de coseno, pero ahora vectorice el texto usando `TF-IDFVectorizer`. Explique el procedimiento que realizó y como ordenó las recomendaciones. Compare con los resultados del punto anterior y explique sus similitudes y/o diferencias.

Se importa TfidfVectorizer de sklearn y el modelo es guardado en un objeto llamado tfidf. Este es aplicado a clean_sentences, de donde se genera una matriz con 520 filas y 24622 columnas (términos)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(clean_sentences)
tfidf_matrix.shap

Esta matriz se convierte a un formato denso almacenado en un data frame denominado df_count que contiene el mismo número de filas y columnas que la matriz anterior. La diferencia con el método CountVectorizer es que en este caso no se generan conteos sino puntajes.

In [ ]:
df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf.get_feature_names_out())

df_tfidf.head()

Se aplica el mismo lematizador realizado previamente:

In [ ]:
def Lematizador_propio(text):
    # Diccionario con las palabras y sus lemas correspondientes
    lemmas = {
        r"\babandona\b": "abandonar",
        r"\babandonado\b": "abandonar",
        r"\babandonandolo\b": "abandonar",
        r"\babandonar\b": "abandonar",
        r"\babandono\b": "abandonar",
        r"\babandón\b": "abandonar",
        r"\babalanzo\b": "abalanzar",
        r"\bzurda\b": "zurdo",
        r"\bzurdazo\b": "zurdo",
        r"\bzurdito\b": "zurdo",
        r"\bzurraba\b": "zurrar",
        r"\bzurrartir\b": "zurrar",
        r"\bzozobra\b": "zozobrar"
    }
    for pattern, lemma in lemmas.items():
        text = re.sub(pattern, lemma, text, flags=re.IGNORECASE)
    return text

clean_sentences2 = list(map(Lematizador_propio, clean_sentences))

Y una vez definido el nuevo lematizador se vuelve a implementar TfidfVectorizer:

In [ ]:
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(clean_sentences2)

df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf.get_feature_names_out())
df_tfidf.head()

Se calcula la distancia de coseno que se guarda en el objeto cosine_sim2 sobre la matriz calculada por TfidfVectorizer:

In [ ]:
cosine_sim2 = linear_kernel(tfidf_matrix, tfidf_matrix)

Para aplicar el recomendador usando la matriz calculada por TfidfVectorizer. Para esto se modifica la opción cosine_sim por cosine_sim2 que contiene las distancias coseno calculadas en la matriz TfidfVectorizer. La ordenación fue realizada dentro de la función recomendador. En la línea sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True) se ordenan los puntajes calculados y en la línea sim_scores = sim_scores[1:6] se extraen los primeros 6. El único cuento que coincide en ambos métodos es el cuento "Gaussian blur"

In [ ]:
recomendador("La venganza del metegol", cosine_sim = cosine_sim2)

A pesar de que los cuentos recomendados fueron diferentes en su mayoría, los términos más repetidos son muy similares a los obtenidos previamente:

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

cuentos_rec=df[df['titulo'].isin(recomendador("La venganza del metegol", cosine_sim = cosine_sim2))]

text = " ".join(cuentos_rec['cuento'])
text = text_cleaning(text)
text=' '.join(text)
text = Lematizador_propio(text)

wordcloud = WordCloud(width = 1600, height = 800, 
    background_color = "white").generate(text)
plt.figure(figsize = (20, 10))
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis("off")
plt.show()

##### 3.1.3. Genere 5 recomendaciones de más recomendada (1) a menos recomendada (5) para el cuento 'La venganza del metegol' usando el texto vectorizado por `TF-IDFVectorizer` y la correlación como medida de similitud. Explique el procedimiento que realizó y como ordenó las recomendaciones. Compare con los resultados de los puntos anteriores y explique sus similitudes y/o diferencias.

In [5]:
# Utilice este espacio para escribir el código.

(Utilice este espacio para describir el procedimiento, análisis, y conclusiones)

##### 3.2. Recomendaciones basadas en temas

Usando modelado de temas con LDA, encuentre los temas subyacentes en el blog. Explique como eligió el numero óptimo de temas. Utilizando el tema asignado al cuento 'La venganza del metegol' y la probabilidad de pertenecer a este tema genere 5 recomendaciones de más recomendada (1) a menos recomendada (5) para este cuento. Explique el procedimiento que realizó. Compare con los resultados encontrados anteriormente y explique sus similitudes y/o diferencias. (Esto puede tomar mucho tiempo y requerir mucha capacidad computacional, puede aprovechar los recursos de [Google Colab](https://colab.research.google.com/))


In [6]:
# Utilice este espacio para escribir el código.

(Utilice este espacio para describir el procedimiento, análisis, y conclusiones)

### 4 Recomendaciones generales

De acuerdo con los resultados encontrados, en su opinión ¿qué procedimiento generó las mejores recomendaciones para la entrada elegida? ¿Cómo implementaría una evaluación objetiva de estas recomendaciones? Justifique su respuesta.

(Utilice este espacio para describir su procedimiento)